# `Offload` a model to MongoDB

In [ ]:

def save_model_to_db(model, client, db, dbconnection, model_type, model_name, accuracy, params):
    import pickle
    from pymongo import MongoClient
    import datetime as dt
    from pprint import pprint as pp

    # pickling the model
    pickled_model = pickle.dumps(model)
    
    # creating connection
    myclient = MongoClient(client)
    # creating database in mongodb
    mydb = myclient[db]
    # creating collection
    mycon = mydb[dbconnection]
    
    # if there is already a model with the same model_type and a worse or equal accuracy, delete it
    mycon.delete_many({"model_type": model_type, "accuracy": {"$lte": accuracy}})
    
    # creating a dictionary to insert
    info = mycon.insert_one({
                                "model": pickled_model,
                                'created_time':dt.datetime.now(),
                                'model_type': model_type,
                                'model_name': model_name,
                                'accuracy': accuracy,
                                'params': params
                                })
    print(info.inserted_id, ' saved with this id successfully!')

    details = {
        'inserted_id':info.inserted_id,
        'created_time':dt.datetime.now(),
        'model_type': model_type,
        'model_name':model_name,
        'accuracy': accuracy,
        'params':params
    }
    pp(details)
    
    return details

### Example on how to use

In [ ]:
model = 'KNeighborsClassifier'
best_accuracy = 0.99
best_parameters = {'n_neighbors':5, 'p':2}

SVM_saved = save_model_to_db(model, 
                             "mongodb://localhost:27017",   # mongodb url
                             'DBName', 
                             'CollectionName',
                             'ModelType', 
                             'ModelName', 
                             best_accuracy, 
                             best_parameters)